In [1]:
pip install cryptography faker pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.7 MB/s eta 0:00:00


AES 256 - Encryptor (Key uses folder format)

In [2]:
import os
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
from faker import Faker
import json
import base64
import secrets
import shutil

# Function to generate a random AES key
def generate_aes_key():
    return secrets.token_bytes(32)  # 32 bytes = 256 bits

# Function to save the encryption key to a file
def save_key(key, filename):
    with open(filename, "wb") as key_file:
        key_file.write(key)

# Function to load the encryption key from a file
def load_key(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Encryption key file '{filename}' not found.")
    with open(filename, "rb") as key_file:
        return key_file.read()

# Step 1: Generate 4 unique AES keys
unique_keys = {
    "admin_key.key": generate_aes_key(),  # Admin key
    "addoc_key.key": generate_aes_key(),  # Shared key for Admin and Doctor
    "all_key.key": generate_aes_key(),    # Shared key for all roles
    "adres_key.key": generate_aes_key()   # Shared key for Admin and Researcher
}

# Save the unique AES keys to files
for key_name, key_value in unique_keys.items():
    save_key(key_value, key_name)

# Step 2: roles are specified with relevant keys to create a folder of aes keys. The keys are copied as per requirement and there exists only 4 unique keys mentioned above.
roles = {
    "Admin": ["admin_key.key", "addoc_key.key", "all_key.key", "adres_key.key"],
    "Doctor": ["addoc_key.key", "all_key.key"],
    "Researcher": ["adres_key.key", "all_key.key"]
}

# Step 3: Create folders and copy keys with role-specific names
for role, keys in roles.items():
    role_folder = f"{role.lower()}_aes_keys"  # e.g., admin_aes_keys
    os.makedirs(role_folder, exist_ok=True)  # Create folder if it doesn't exist
    for key in keys:
        # Copy the key into the role folder with a role-specific name
        role_specific_key_name = f"{role.lower()}_{key}"  # e.g., admin_admin_key.key
        shutil.copy(key, os.path.join(role_folder, role_specific_key_name))

# Step 4: Delete the original individual key files
for key_name in unique_keys.keys():
    if os.path.exists(key_name):
        os.remove(key_name)  # Delete the original key file

# Step 5: Load the keys for encryption
# Load the keys from the unique key files
key_files = {key_name: load_key(os.path.join(f"{role.lower()}_aes_keys", f"{role.lower()}_{key_name}")) for role, keys in roles.items() for key_name in keys if os.path.exists(os.path.join(f"{role.lower()}_aes_keys", f"{role.lower()}_{key_name}"))}

# Function to initialize AES cipher
def initialize_cipher(key, iv=None):
    if iv is None:
        iv = os.urandom(16)  # Generate a random 16-byte IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    return cipher, iv

# Function to encrypt data using AES-256
def aes_encrypt(data, key):
    if data is None or pd.isnull(data):  # Handle null or missing values
        data = "None"  # Replace with a placeholder string
    cipher, iv = initialize_cipher(key)
    encryptor = cipher.encryptor()
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data.encode()) + padder.finalize()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return base64.b64encode(iv + encrypted_data).decode()  # Encode IV + ciphertext in Base64

# Load the dataset
try:
    df = pd.read_csv("patient_raw_data.csv")
except FileNotFoundError:
    raise FileNotFoundError("The file 'patient_raw_data.csv' was not found. Please provide the dataset.")

# Step 6: Pseudonymize name and encrypt using admin key. Map original names and pseudonymized name using json file.
#Initialize Faker for pseudonymization
faker = Faker()

# Create a dictionary to store the mapping between original and pseudonymized names
name_mapping = {}

# Function to pseudonymize and encrypt names ----(Admin key)------
def pseudonymize_and_encrypt_name(name, admin_key):
    if name is None or pd.isnull(name):  # Handle null or missing values
        name = "None"  # Replace with a placeholder string
    try:
        # Check if the name is already pseudonymized
        if name in name_mapping:
            pseudonymized_name = name_mapping[name]
        else:
            # Generate a pseudonymized name
            pseudonymized_name = faker.name()
            # Store the mapping between the original and pseudonymized name
            name_mapping[name] = pseudonymized_name
        # Encrypt the pseudonymized name using the admin's key
        encrypted_name = aes_encrypt(pseudonymized_name, admin_key)
        return encrypted_name
    except Exception as e:
        print(f"Error pseudonymizing and encrypting name '{name}': {e}")
        return None

# Function to encrypt any field with a specific key
def encrypt_field(value, key):
    if value is None or pd.isnull(value):  # Handle null or missing values
        value = "None"  # Replace with a placeholder string
    return aes_encrypt(str(value), key)  # Ensure all data is treated as strings

# Define role-based access with keys. Each column is assigned with an unique key
role_based_access = {
    "Admin": ["Name", "DOB", "Patient Address"],  # Admin key is assigned with 3 unique columns
    "AdDoc": ["Patient ID", "Patient Complaint", "Lab Results", "Physician Notes", "Next Follow-up Date"],  # AdDoc key is assigned with 5 unique columns
    "All": ["Age", "Gender", "Ailment", "Family Medical History", "Negative Lifestyle Factor", "Treatment", "Medications"],  # All key is assigned with 7 columns
    "AdRes": ["Genetic Predisposition"]  # AdRes key is assigned with 1 column
}

#Step 7: Encrypt columns based on keys
for role, columns in role_based_access.items():
    key = None
    if role == "Admin":
        key = key_files["admin_key.key"]
    elif role == "AdDoc":
        key = key_files["addoc_key.key"]
    elif role == "All":
        key = key_files["all_key.key"]
    elif role == "AdRes":
        key = key_files["adres_key.key"]

    for col in columns:
        if col in df.columns and col != "Name":  # Skip "Name" column for now
            df[col] = df[col].apply(lambda x: encrypt_field(x, key))

#Step 8: Pseudonymize and encrypt the 'Name' column (Admin key)
if "Name" in df.columns:
    df["Name"] = df["Name"].apply(lambda x: pseudonymize_and_encrypt_name(x, key_files["admin_key.key"]))

# Save the preprocessed dataset and the name mapping
df.to_csv("encrypted_dataset.csv", index=False)
with open("name_mapping.json", "w") as f:
    json.dump(name_mapping, f, indent=4)  # Save the mapping as a JSON file with indentation for readability

# Output files created:
#print("Encrypted Dataset with AES-256:")
#print(df.head())  # Display the first few rows of the encrypted dataset
print("\n===AES 256 encryptor===\n")
print("Processing dataset..\n")
print("Dataset has been processed and saved as 'encrypted_dataset.csv'")
print("The keys are saved and stored in role specific folders.")

Encrypted Dataset with AES-256:
                                     Patient ID  \
0  Jhij900uILlvOb0CaA25YezJj6ta5XmbUHsSk1LsmyU=   
1  c5q1uamPQzOKMFI8fWKnBDipCTnwbOvaULOea7qdvm8=   
2  hm6OTARsVQUV5KaGLcwIVZ0jndHMyPpjr6aVYoECnj0=   
3  iB7ajoX1W5UHM5X/heLeslcMHEQKNgbTKecBTQ3TA/o=   
4  YkqrU/jNnYSIwRfjzw/XCG5MHXDS4pkGjtWoYTd+C2w=   

                                           Name  \
0  Q303sHgRKCmMsOOBwHGHRVrps5BEsbLBkGBcjCMFDys=   
1  To5fxqMjTnLt2Lp0975DSPWiY9I3wbq6WyF1Cv6QY+w=   
2  MQIM9OrkowsekYz8n00ayUXxdaTFNv/f8HfWOT9nfCY=   
3  BNbVbZ9q3XzllRipMl85tVdmx7FsBqmsVeSTg2zeuBc=   
4  R6dSq2pA9ljCFF4Z+qp20D10PT6tyxvpdubXp5/NE8k=   

                                            DOB  \
0  fo9LjqgMJeKLDrndkYdGeJWg+L7Ii85/k5rt55nkSzs=   
1  M55iheqEH1+zuIUoaNqM9LtUk/JW2kWOOc/u5PvKOTY=   
2  poZjA+ebNwEBrtwmfnho0seGGqpL452bwDyMmNLLw6E=   
3  uor8iwNji0hkQmXZQ2sfBB/h4hsWO8TMs7h7YfPlqu4=   
4  txFmwNympI/hIjMHWY/NzZx39TWpk0uKLARxB41y09g=   

                                            Age

RSA key generation for AES key folders.

In [3]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
import os
import json

def generate_rsa_keys(role):
    # Generate RSA key pair
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # Create keys directory if it doesn't exist
    if not os.path.exists('rsa_keys'):
        os.makedirs('rsa_keys')

    # Save private key
    private_key_path = f"rsa_keys/{role}_private_key.pem"
    with open(private_key_path, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # Save public key
    public_key_path = f"rsa_keys/{role}_public_key.pem"
    with open(public_key_path, "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

    return public_key

def encrypt_file(file_path, public_key):
    with open(file_path, 'rb') as file:
        data = file.read()

    # Encrypt the data
    encrypted_data = public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Save encrypted data
    encrypted_path = file_path + '.encrypted'
    with open(encrypted_path, 'wb') as file:
        file.write(encrypted_data)

    return encrypted_path

def encrypt_folder(folder_path, public_key):
    encrypted_files = {}

    # Walk through the folder
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.key'):
                file_path = os.path.join(root, file)
                encrypted_path = encrypt_file(file_path, public_key)
                encrypted_files[file] = os.path.basename(encrypted_path)
                # Remove original file
                os.remove(file_path)

    return encrypted_files

print("=== AES Key Folder Encryption System ===")

def main():
    # Define roles and their corresponding folders
    role_folders = {
        'admin': 'admin_aes_keys',
        'doctor': 'doctor_aes_keys',
        'researcher': 'researcher_aes_keys'
    }

    # Store encryption mapping
    encryption_mapping = {}

    # Generate separate RSA keys and encrypt each folder
    for role, folder in role_folders.items():
        if os.path.exists(folder):
            print(f"Processing {role}'s folder...")
            # Generate unique RSA keys for this role
            public_key = generate_rsa_keys(role)
            # Encrypt the folder
            encryption_mapping[folder] = encrypt_folder(folder, public_key)

    # Save encryption mapping
    with open('encryption_mapping.json', 'w') as f:
        json.dump(encryption_mapping, f, indent=4)

if __name__ == "__main__":
    main()

# Created/Modified files during execution:
print("\nCreated files:")
print("- rsa_keys/admin_private_key.pem")
print("- rsa_keys/admin_public_key.pem")
print("- rsa_keys/doctor_private_key.pem")
print("- rsa_keys/doctor_public_key.pem")
print("- rsa_keys/researcher_private_key.pem")
print("- rsa_keys/researcher_public_key.pem")
print("- encryption_mapping.json")
print("- *.key.encrypted files in each folder")

=== AES Key Folder Encryption System ===
Processing admin's folder...
Processing doctor's folder...
Processing researcher's folder...

Created files:
- rsa_keys/admin_private_key.pem
- rsa_keys/admin_public_key.pem
- rsa_keys/doctor_private_key.pem
- rsa_keys/doctor_public_key.pem
- rsa_keys/researcher_private_key.pem
- rsa_keys/researcher_public_key.pem
- encryption_mapping.json
- *.key.encrypted files in each folder


Decrypt RSA keys

In [4]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend
import os
import json
from datetime import datetime

def load_private_key(role):
    private_key_path = f"rsa_keys/{role}_private_key.pem"
    if not os.path.exists(private_key_path):
        raise FileNotFoundError(f"Private key not found for role: {role}")

    with open(private_key_path, "rb") as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            password=None,
            backend=default_backend()
        )
    return private_key

def decrypt_file(encrypted_file_path, private_key):
    with open(encrypted_file_path, 'rb') as file:
        encrypted_data = file.read()

    # Decrypt the data
    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Save decrypted data
    decrypted_path = encrypted_file_path.replace('.encrypted', '')
    with open(decrypted_path, 'wb') as file:
        file.write(decrypted_data)

    return decrypted_path

def log_access(name, role):
    log_dir = 'access_logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} - User: {name}, Role: {role} accessed their keys\n"

    with open(f"{log_dir}/access_log.txt", 'a') as f:
        f.write(log_entry)

def decrypt_folder_by_role(name, role):
    try:
        print(f"\nProcessing request for {name} with role: {role}")

        # Load private key for this role
        private_key = load_private_key(role)

        # Load encryption mapping
        with open('encryption_mapping.json', 'r') as f:
            encryption_mapping = json.load(f)

        folder_name = f"{role}_aes_keys"
        if folder_name not in encryption_mapping:
            print(f"No encrypted files found for role: {role}")
            return

        # Create folder if it doesn't exist
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Decrypt files for the role
        decrypted_files = []
        for original_file, encrypted_file in encryption_mapping[folder_name].items():
            encrypted_path = os.path.join(folder_name, encrypted_file)
            if os.path.exists(encrypted_path):
                decrypted_path = decrypt_file(encrypted_path, private_key)
                decrypted_files.append(os.path.basename(decrypted_path))
                print(f"Decrypted: {decrypted_path}")
                # Remove encrypted file
                os.remove(encrypted_path)
            else:
                print(f"Encrypted file not found: {encrypted_path}")

        # Log the access
        log_access(name, role)

        if decrypted_files:
            print(f"\nSuccessfully decrypted the following files for {name}:")
            for file in decrypted_files:
                print(f"- {file}")
        print(f"\nFiles are available in the {folder_name} folder")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    print("=== AES Key Folder Decryption System ===")

    # Get user details
    name = input("Please enter your name: ").strip()
    while not name:
        print("Name cannot be empty.")
        name = input("Please enter your name: ").strip()

    # Get role with validation
    valid_roles = ['admin', 'doctor', 'researcher']
    while True:
        role = input("Enter your role (admin/doctor/researcher): ").lower().strip()
        if role in valid_roles:
            break
        print("Invalid role. Please enter admin, doctor, or researcher.")

    # Perform decryption
    decrypt_folder_by_role(name, role)

if __name__ == "__main__":
    main()

# Created/Modified files during execution:
print("\nCreated files:")
print("- Decrypted *.key files in respective role folders")
print("- access_logs: access_log.txt")

=== AES Key Folder Decryption System ===
Please enter your name: Bhushan
Enter your role (admin/doctor/researcher): researcher

Processing request for Bhushan with role: researcher
Decrypted: researcher_aes_keys/researcher_adres_key.key
Decrypted: researcher_aes_keys/researcher_all_key.key

Successfully decrypted the following files for Bhushan:
- researcher_adres_key.key
- researcher_all_key.key

Files are available in the researcher_aes_keys folder

Created/Modified files:
- Decrypted *.key files in respective role folders
- access_logs/access_log.txt


AES 256 - Decryptor

In [8]:
import os
import pandas as pd
import json
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

# Function to load the encryption key from a file
def load_key(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Encryption key file '{filename}' not found.")
    with open(filename, "rb") as key_file:
        key = key_file.read()
    if len(key) != 32:  # Ensure the key is 32 bytes
        raise ValueError(f"Invalid key size in '{filename}'. Expected 32 bytes, got {len(key)} bytes.")
    return key

# Function to initialize AES cipher
def initialize_cipher(key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    return cipher

# Function to decrypt data using AES-256
def aes_decrypt(encrypted_data, key):
    if not encrypted_data:
        return None
    try:
        encrypted_data = base64.b64decode(encrypted_data)
        iv = encrypted_data[:16]  # Extract the IV (first 16 bytes)
        cipher = initialize_cipher(key, iv)
        decryptor = cipher.decryptor()
        padded_data = decryptor.update(encrypted_data[16:]) + decryptor.finalize()
        unpadder = PKCS7(algorithms.AES.block_size).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()
        return data.decode()
    except Exception as e:
        print(f"Decryption error: {e}")
        return None

# Function to decrypt the 'Name' column and map it to the original name
def decrypt_and_map_name(encrypted_name, key, reversed_name_mapping):
    if not encrypted_name:
        return None
    try:
        # Step 1: Decrypt the cipher text to retrieve the pseudonymized name
        pseudonymized_name = aes_decrypt(encrypted_name, key)
        if pseudonymized_name is None:
            return None

        # Step 2: Map the pseudonymized name to the original name
        original_name = reversed_name_mapping.get(pseudonymized_name)
        if original_name:
            return original_name
        else:
            print(f"Warning: No mapping found for pseudonymized name '{pseudonymized_name}'.")
            return pseudonymized_name  # Return the pseudonymized name if no mapping is found
    except Exception as e:
        print(f"Error decrypting and mapping name: {e}")
        return None

# Role-based access configuration
role_based_access = {
    "admin" : ["Name", "DOB", "Patient Address", "Age", "Gender", "Ailment", "Family Medical History",
              "Negative Lifestyle Factor", "Treatment", "Medications", "Genetic Predisposition",
              "Patient ID", "Patient Complaint", "Lab Results", "Physician Notes", "Next Follow-up Date"],
    "doctor": ["Age", "Gender", "Ailment", "Family Medical History", "Negative Lifestyle Factor",
               "Treatment", "Medications", "Patient ID", "Patient Complaint", "Lab Results",
               "Physician Notes", "Next Follow-up Date"],
    "researcher": ["Age", "Gender", "Ailment", "Family Medical History", "Negative Lifestyle Factor",
                   "Treatment", "Medications", "Genetic Predisposition"]
}

# Function to load keys dynamically from the role-specific folder
def load_keys_from_folder(role_folder, user_role):
    keys = {}
    key_mapping = {
        f"{user_role}_admin_key.key": "admin_key",
        f"{user_role}_all_key.key": "all_key",
        f"{user_role}_addoc_key.key": "addoc_key",
        f"{user_role}_adres_key.key": "adres_key"
    }

    for key_file in os.listdir(role_folder):
        key_path = os.path.join(role_folder, key_file)
        if key_file in key_mapping:
            key_name = key_mapping[key_file]
            keys[key_name] = load_key(key_path)
        else:
            print(f"Warning: Unrecognized key file '{key_file}' in folder '{role_folder}'. Skipping.")
    return keys

# Function to decrypt the dataset based on the user's role
def decrypt_dataset(user_name, user_role, encrypted_file, name_mapping_file, output_file):
    # Map roles to their respective folders
    role_folders = {
        "admin": "admin_aes_keys",
        "doctor": "doctor_aes_keys",
        "researcher": "researcher_aes_keys"
    }

    # Check if the role folder exists
    role_folder = role_folders.get(user_role.lower())
    if not role_folder or not os.path.exists(role_folder):
        raise ValueError(f"Invalid role '{user_role}' or folder '{role_folder}' does not exist.")

    # Load the keys from the role folder
    keys = load_keys_from_folder(role_folder, user_role)

    # Load the encrypted dataset
    try:
        df = pd.read_csv(encrypted_file)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{encrypted_file}' was not found. Please provide the dataset.")

    # Load the name mapping file
    try:
        with open(name_mapping_file, "r") as f:
            name_mapping = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{name_mapping_file}' was not found. Please provide the mapping file.")

    # Reverse the name mapping to map pseudonymized names back to original names
    reversed_name_mapping = {v: k for k, v in name_mapping.items()}

    # Get the columns the user is allowed to decrypt
    allowed_columns = role_based_access.get(user_role.lower())
    if not allowed_columns:
        raise ValueError(f"Invalid role '{user_role}'. Valid roles are: {', '.join(role_based_access.keys())}.")

    # Decrypt the dataset
    for col in df.columns:
        if col in allowed_columns:
            # Determine the appropriate key for the column
            if col in ["Name", "DOB", "Patient Address"]:
                key = keys["admin_key"]
            elif col in ["Age", "Gender", "Ailment", "Family Medical History", "Negative Lifestyle Factor",
                         "Treatment", "Medications"]:
                key = keys["all_key"]
            elif col in ["Genetic Predisposition"]:
                key = keys["adres_key"]
            elif col in ["Patient ID", "Patient Complaint", "Lab Results", "Physician Notes", "Next Follow-up Date"]:
                key = keys["addoc_key"]
            else:
                continue  # Skip columns not explicitly mapped

            # Decrypt the column
            if col == "Name":  # Special handling for the 'Name' column
                df[col] = df[col].apply(lambda x: decrypt_and_map_name(x, key, reversed_name_mapping) if pd.notnull(x) else None)
            else:
                df[col] = df[col].apply(lambda x: aes_decrypt(x, key) if pd.notnull(x) else None)

    # Save the decrypted dataset to a new CSV file
    df.to_csv(output_file, index=False)
    print(f"Decrypted dataset saved to '{output_file}'.")

# Main function to prompt user and decrypt dataset
def main():
    try:
        # Prompt user for their name and role
        user_name = input("Enter your name: ").strip()
        user_role = input("Enter your role (Admin, Doctor, Researcher): ").strip().capitalize()

        # Check if the role is valid
        valid_roles = ["Admin", "Doctor", "Researcher"]
        if user_role not in valid_roles:
            raise ValueError(f"Invalid role '{user_role}'. Valid roles are: {', '.join(valid_roles)}.")

        # Greet the user with their role
        print(f"Hello, {user_name}! You are accessing the system as a '{user_role}'.")

        # Specify the encrypted file, name mapping file, and output file
        encrypted_file = "encrypted_dataset.csv"
        name_mapping_file = "name_mapping.json"
        output_file = f"decrypted_data_{user_role.lower()}.csv"

        # Decrypt the dataset
        decrypt_dataset(user_name, user_role.lower(), encrypted_file, name_mapping_file, output_file)
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    main()

Enter your name: Bhushan
Enter your role (Admin, Doctor, Researcher): researcher
Hello, Bhushan! You are accessing the system as a 'Researcher'.
Decrypted dataset saved to 'decrypted_data_researcher.csv'.
